In [1]:
import sys
import pandas as pd
import numpy as np
import difflib
import gzip
from math import radians, cos, sin, asin, sqrt

In [2]:
filename1 = "stations.json.gz"
filename2 = "city_data.csv"
filename3 = "output.svg" 
# filename1 = sys.argv[1]
# filename2 = sys.argv[2]
# filename3 = sys.argv[3]

In [3]:
# Calculate distance from lat/lon. Copied from stackoverflow.com
# https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
#     r = r * 1000 # convert to meters
    return c * r

In [4]:
station_fh = gzip.open(filename1, 'rt', encoding='utf-8')
stations = pd.read_json(station_fh, lines=True)
cities = pd.read_table(filename2,sep=',')
# devide by 10 on temperature
stations["avg_tmax"] = stations["avg_tmax"].apply(lambda x: x/10)

# Exclude cities with area > 10000 sq km
cities["area"] = cities["area"].apply(lambda a : a /1000000.0)
cities = cities[cities["area"]<=10000]
stations

,avg_tmax,elevation,latitude,longitude,observations,station
0,10.201667,691.0,54.4500,-124.2833,300,CA001092970
1,13.930937,967.0,50.0333,-113.2167,320,CA003030529
2,12.355311,902.0,49.0000,-108.3833,273,CA004038116
3,14.597727,16.0,46.5833,-72.2333,220,CA007016840
4,21.910429,190.8,34.5128,-93.0486,326,USC00033466
5,17.921788,1864.5,39.4286,-105.0703,358,USC00057249
6,25.847268,94.8,32.7128,-82.5414,366,USC00094862
7,15.010929,722.7,48.3511,-116.8353,366,USC00107386
8,14.479508,1501.1,43.8867,-111.7367,366,USC00108818
9,9.858356,185.9,47.2386,-68.6136,365,USC00172878


In [ ]:
cities

,name,population,area,latitude,longitude
2,Calgary,1096833.0,825.290000,51.054444,-114.066944
6,Edmonton,812201.0,684.370000,53.500000,-113.500000
18,Abbotsford,133497.0,375.550000,49.054611,-122.328000
20,Burnaby,223218.0,90.610000,49.250000,-122.949167
27,Cranbrook,NaN,31.950000,49.513600,-115.769000
42,Nanaimo,83811.0,91.000000,49.164167,-123.936389
56,Revelstoke,7139.0,40.000000,50.998100,-118.196000
57,Richmond,198309.0,129.270000,49.166667,-123.133333
60,Surrey,468251.0,316.410000,49.183300,-122.850000
63,Vancouver,603502.0,115.000000,49.250000,-123.100000


In [ ]:
def distance(city, stations):
#     print(city)
    lat1 = city["latitude"]
    lon1 = city["longitude"]
    stations.
    return stations.apply(lambda x: haversine(lat1,lon1, x["latitude"], x["longitude"]),axis=1)

def best_tmax(city, stations):
    print("xx")
    distances = distance(city, stations)
    city_stations=stations.copy()
    city_stations["dist"] = distances
    min_station = city_stations.iloc[city_stations["dist"].idxmin()]    
    return min_station["avg_tmax"] 

# a = distance(cities.iloc[0],stations)
cities.apply(distance, stations=stations,axis=1)
#distance(cities.iloc[0],stations)
# best_tmax(cities.iloc[-1],stations)